In [ ]:
!pip install -U langchain-ollama

In [ ]:
# !pip install langchain-community
# !pip install pdfplumber
# !pip install langchain-experimental
!pip install sentence-transformers

In [ ]:
from langchain_community.document_loaders import PDFPlumberLoader
loader = PDFPlumberLoader(r"C:\Users\Mohit\Downloads\xtractlylangchain\xtractlylangchain\demo\notebooks\mortgage.pdf")
docs = loader.load()

# Check the number of pages
print("Number of pages in the PDF:",len(docs))

# Load the random page content
# docs[0].page_content

In [ ]:
!pip list

In [ ]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain.embeddings import HuggingFaceEmbeddings

text_splitter = SemanticChunker(HuggingFaceEmbeddings())
documents = text_splitter.split_documents(docs)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

template = """Question: {question}

Answer: Let's think step by step."""

prompt = ChatPromptTemplate.from_template(template)

model = OllamaLLM(model="llama3.2")

chain = prompt | model

chain.invoke({"question": "What is LangChain?"})

In [ ]:
# !pip install gradio
!pip install faiss-cpu

In [ ]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Ollama
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains import RetrievalQA
import gradio as gr

# Load the PDF
loader = PDFPlumberLoader(r"C:\Users\Mohit\Downloads\xtractlylangchain\xtractlylangchain\demo\notebooks\mortgage.pdf")
docs = loader.load()

# Split into chunks
text_splitter = SemanticChunker(HuggingFaceEmbeddings())
documents = text_splitter.split_documents(docs)


# Instantiate the embedding model
embedder = HuggingFaceEmbeddings()

# Create the vector store and fill it with embeddings
vector = FAISS.from_documents(documents, embedder)
retriever = vector.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# Define llm
llm = Ollama(model="llama3.2")

# Define the prompt
prompt = """
1. Use the following pieces of context to answer the question at the end.
2. If you don't know the answer, just say that "I don't know" but don't make up an answer on your own.\n
3. Keep the answer crisp with labels.

Context: {context}

Question: {question}

Helpful Answer:"""

QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt) 

llm_chain = LLMChain(
                  llm=llm, 
                  prompt=QA_CHAIN_PROMPT, 
                  callbacks=None, 
                  verbose=True)

document_prompt = PromptTemplate(
    input_variables=["page_content", "source"],
    template="Context:\ncontent:{page_content}\nsource:{source}",
)

combine_documents_chain = StuffDocumentsChain(
                  llm_chain=llm_chain,
                  document_variable_name="context",
                  document_prompt=document_prompt,
                  callbacks=None)
              
qa = RetrievalQA(
                  combine_documents_chain=combine_documents_chain,
                  verbose=False,
                  retriever=retriever,
                  return_source_documents=True)



C:\Users\Mohit\AppData\Local\Temp\ipykernel_4084\877135553.py:17: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  text_splitter = SemanticChunker(HuggingFaceEmbeddings())
C:\Users\Mohit\AppData\Local\Temp\ipykernel_4084\877135553.py:22: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embedder = HuggingFaceEmbeddings()


In [17]:
print(qa("what is the type of the document and BUYER INFORMATION?")["result"])



> Entering new RetrievalQA chain...


> Entering new LLMChain chain...
Prompt after formatting:

1. Use the following pieces of context to answer the question at the end.
2. If you don't know the answer, just say that "I don't know" but don't make up an answer on your own.

3. Keep the answer crisp with labels.

Context: Context:
content:BUYER INFORMATION
First Name: John
Last Name: Doe
Address: 123 Palm Tree Lane
City: Miami
State: FL
Zipcode: 33101
SELLER INFORMATION
First Name: Jane
Last Name: Smith
Address: 456 Sunny Isles Boulevard
City: Miami
State: FL

source:C:\Users\Mohit\Downloads\xtractlylangchain\xtractlylangchain\demo\notebooks\mortgage.pdf

Context:
content:MORTGAGE DEED
This Mortgage Deed, made this 1st day of February, 2024, between John Doe
(hereinafter referred to as the "Mortgagor"), residing at 123 Palm Tree Lane, Miami,
FL 33101, and Jane Smith (hereinafter referred to as the "Mortgagee"), residing at
456 Sunny Isles Boulevard, Miami, FL 33160, witnesseth:
PROPER